In [1]:
from tqdm import tqdm
import time
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
#import tick customization tools
import matplotlib.ticker as mticks
import matplotlib.dates as mdates
## Setting figures to timeseries-friendly
plt.rcParams['figure.figsize'] = (12,4)
plt.rcParams['figure.facecolor'] = 'white'
sns.set_context("talk", font_scale=0.9)
# set random seed
SEED = 321
np.random.seed(SEED)
#display more columns
pd.set_option('display.max_columns',50)



# Loading Data

In [2]:
folder = "Data/Chicago/"
crime_files = sorted(glob.glob(folder+"*.csv"))
chicago_df = pd.concat([pd.read_csv(f) for f in crime_files])
chicago_df

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,1358218,01/01/2001 01:00:00 PM,ASSAULT,SIMPLE,APARTMENT,False,False,312,3.0,NaN,41.778744,-87.612135
1,1310288,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,621,6.0,NaN,41.756650,-87.641608
2,1310393,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1614,16.0,NaN,41.974911,-87.846348
3,1316324,01/01/2001 01:00:00 PM,THEFT,OVER $500,STREET,False,False,1513,15.0,NaN,41.869008,-87.773947
4,1311626,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1033,10.0,NaN,41.848786,-87.704087
...,...,...,...,...,...,...,...,...,...,...,...,...
123545,13124429,06/30/2023 12:50:00 AM,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,1925,19.0,46.0,41.949093,-87.643737
123546,13124821,06/30/2023 12:50:00 PM,THEFT,RETAIL THEFT,TAVERN / LIQUOR STORE,False,False,1924,19.0,44.0,41.939906,-87.654410
123547,13124281,06/30/2023 12:52:00 AM,SEX OFFENSE,PUBLIC INDECENCY,STREET,False,False,2534,25.0,36.0,41.909763,-87.735792
123548,13124301,06/30/2023 12:57:00 AM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,832,8.0,18.0,41.761952,-87.684388


# Chicago Dataframe

In [3]:
chicago_df.isna().sum()

ID                           0
Date                         0
Primary Type                 0
Description                  0
Location Description     10687
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614849
Latitude                 88034
Longitude                88034
dtype: int64

In [4]:
chicago_df.dropna(inplace = True)
chicago_df

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
8,6154338,01/01/2001 01:00:00 PM,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,831,8.0,15.0,41.774819,-87.702896
10,3769790,01/01/2001 01:00:00 PM,THEFT,AGG: FINANCIAL ID THEFT,RESIDENCE,False,False,523,5.0,34.0,41.673171,-87.638000
13,3206463,01/01/2001 01:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,835,8.0,18.0,41.735507,-87.690095
22,3212105,01/01/2001 01:00:00 AM,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,True,False,913,9.0,14.0,41.823730,-87.698440
44,5462733,01/01/2001 01:00:00 AM,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,233,2.0,20.0,41.789084,-87.620849
...,...,...,...,...,...,...,...,...,...,...,...,...
123545,13124429,06/30/2023 12:50:00 AM,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,1925,19.0,46.0,41.949093,-87.643737
123546,13124821,06/30/2023 12:50:00 PM,THEFT,RETAIL THEFT,TAVERN / LIQUOR STORE,False,False,1924,19.0,44.0,41.939906,-87.654410
123547,13124281,06/30/2023 12:52:00 AM,SEX OFFENSE,PUBLIC INDECENCY,STREET,False,False,2534,25.0,36.0,41.909763,-87.735792
123548,13124301,06/30/2023 12:57:00 AM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,832,8.0,18.0,41.761952,-87.684388


In [5]:
chicago_df['Datetime'] = pd.to_datetime(chicago_df['Date'], format="%m/%d/%Y %H:%M:%S %p")
chicago_df = chicago_df.sort_values('Datetime')
chicago_df = chicago_df.set_index('Datetime')
chicago_df.info()
chicago_df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7135456 entries, 2001-01-01 01:00:00 to 2023-06-30 12:58:00
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Latitude              float64
 11  Longitude             float64
dtypes: bool(2), float64(4), int64(2), object(4)
memory usage: 612.4+ MB


,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
Datetime,,,,,,,,,,,,
2001-01-01 01:00:00,6154338,01/01/2001 01:00:00 PM,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,831,8.0,15.0,41.774819,-87.702896
2001-01-01 01:00:00,3769790,01/01/2001 01:00:00 PM,THEFT,AGG: FINANCIAL ID THEFT,RESIDENCE,False,False,523,5.0,34.0,41.673171,-87.638000
2001-01-01 01:00:00,3206463,01/01/2001 01:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,835,8.0,18.0,41.735507,-87.690095
2001-01-01 01:00:00,3212105,01/01/2001 01:00:00 AM,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,True,False,913,9.0,14.0,41.823730,-87.698440
2001-01-01 01:00:00,5462733,01/01/2001 01:00:00 AM,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,233,2.0,20.0,41.789084,-87.620849


## Holiday Dataframe

In [6]:
crime_files = sorted(glob.glob("Data/"+"*.csv"))
holiday_df = pd.concat([pd.read_csv(f) for f in crime_files])
holiday_df

,Date,US Holidays,State Holidays
0,2001-01-01,New Year's Day,New Year's Day
1,2001-01-02,NaN,NaN
2,2001-01-03,NaN,NaN
3,2001-01-04,NaN,NaN
4,2001-01-05,NaN,NaN
...,...,...,...
7942,2022-09-30,NaN,NaN
7943,2022-10-01,NaN,NaN
7944,2022-10-02,NaN,NaN
7945,2022-10-03,NaN,NaN


In [7]:
# nulls
holiday_df.isna().sum()

Date                 0
US Holidays       7703
State Holidays    7646
dtype: int64

In [8]:
holiday_df.dropna(inplace = True)
holiday_df.drop(columns = ["State Holidays"], inplace = True)
holiday_df

,Date,US Holidays
0,2001-01-01,New Year's Day
14,2001-01-15,Martin Luther King Jr. Day
49,2001-02-19,Washington's Birthday
147,2001-05-28,Memorial Day
184,2001-07-04,Independence Day
...,...,...
7819,2022-05-30,Memorial Day
7839,2022-06-19,Juneteenth National Independence Day
7840,2022-06-20,Juneteenth National Independence Day (Observed)
7854,2022-07-04,Independence Day


In [47]:
!pip install holidays

     -------------------------------------- 642.9/642.9 kB 8.0 MB/s eta 0:00:00


In [48]:
import holidays
import datetime as dt
from holidays import country_holidays

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [50]:
## making a date range that covers full dataset
all_days = pd.date_range(chicago_df["Date"].min(), chicago_df["Date"].max())
all_days



DatetimeIndex(['2001-01-01 01:00:00', '2001-01-02 01:00:00',
               '2001-01-03 01:00:00', '2001-01-04 01:00:00',
               '2001-01-05 01:00:00', '2001-01-06 01:00:00',
               '2001-01-07 01:00:00', '2001-01-08 01:00:00',
               '2001-01-09 01:00:00', '2001-01-10 01:00:00',
               ...
               '2022-12-22 01:00:00', '2022-12-23 01:00:00',
               '2022-12-24 01:00:00', '2022-12-25 01:00:00',
               '2022-12-26 01:00:00', '2022-12-27 01:00:00',
               '2022-12-28 01:00:00', '2022-12-29 01:00:00',
               '2022-12-30 01:00:00', '2022-12-31 01:00:00'],
              dtype='datetime64[ns]', length=8035, freq='D')

In [51]:
## Create an instance of the US country holidays.
us_holidays = country_holidays('US')
us_holidays

holidays.country_holidays('US')

In [52]:
## Testing first date
print(all_days[0])
us_holidays.get(all_days[0])

2001-01-01 01:00:00


"New Year's Day"

In [53]:
## Getting us holidays for all dates
holiday_list = [us_holidays.get(day) for day in all_days]
holiday_list[:5]

["New Year's Day", None, None, None, None]

In [60]:
# For a specific subdivisions (e.g. state or province):
co_holidays = country_holidays('US', subdiv='CO')
co_holidays

holidays.country_holidays('US', subdiv='CO')

In [61]:
## Saving both holiday types as columns
chicago_df["US Holiday"] = [us_holidays.get(day) for day in chicago_df['Date']]
chicago_df['CO Holiday'] = [co_holidays.get(day) for day in chicago_df['Date']]
chicago_df.head()

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude,Total # of Crimes,US Holiday,CO Holiday
Datetime,,,,,,,,,,,,,,,
2001-01-01 01:00:00,6154338,01/01/2001 01:00:00 PM,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,831,8.0,15.0,41.774819,-87.702896,NaN,New Year's Day,New Year's Day
2001-01-01 01:00:00,3769790,01/01/2001 01:00:00 PM,THEFT,AGG: FINANCIAL ID THEFT,RESIDENCE,False,False,523,5.0,34.0,41.673171,-87.638000,NaN,New Year's Day,New Year's Day
2001-01-01 01:00:00,3206463,01/01/2001 01:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,835,8.0,18.0,41.735507,-87.690095,NaN,New Year's Day,New Year's Day
2001-01-01 01:00:00,3212105,01/01/2001 01:00:00 AM,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,True,False,913,9.0,14.0,41.823730,-87.698440,NaN,New Year's Day,New Year's Day
2001-01-01 01:00:00,5462733,01/01/2001 01:00:00 AM,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,233,2.0,20.0,41.789084,-87.620849,NaN,New Year's Day,New Year's Day


In [ ]:
## US Holidays
chicago_df['US Holiday'].value_counts()

In [ ]:
## MD Holidays
chicago_df['CO Holiday'].value_counts()

In [ ]:
ax = sns.barplot(data= chicago_dfgo_df, x='US Holiday',y='Total_Incidents',estimator=np.sum)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right');

# Resampled Dataframe

In [15]:
## Creating a Total # of Crimes
# getting list of  unique crime descriptions
crime_list = chicago_df['Description'].unique()
crime_list

array(['FINANCIAL ID THEFT: OVER $300', 'AGG: FINANCIAL ID THEFT',
       'PREDATORY', 'AGG SEX ASSLT OF CHILD FAM MBR',
       'AGG CRIM SEX ABUSE FAM MEMBER', 'OVER $500',
       'AGG CRIMINAL SEXUAL ABUSE', 'FIRST DEGREE MURDER',
       'THEFT/RECOVERY: AUTOMOBILE', 'CRIM SEX ABUSE BY FAM MEMBER',
       'SEXUAL EXPLOITATION OF A CHILD', 'AUTOMOBILE',
       'OTHER VEHICLE OFFENSE', 'FRAUD OR CONFIDENCE GAME',
       'SEX ASSLT OF CHILD BY FAM MBR', 'CREDIT CARD FRAUD',
       '$500 AND UNDER', 'FINANCIAL ID THEFT:$300 &UNDER',
       'AGGRAVATED: OTHER', 'NON-AGGRAVATED', 'EMBEZZLEMENT', 'FORGERY',
       'HARASSMENT BY ELECTRONIC MEANS', 'BOGUS CHECK', 'CHILD ABUSE',
       'FROM BUILDING', 'CONTRIBUTE DELINQUENCY OF A CHILD',
       'HARASSMENT BY TELEPHONE', 'FINAN EXPLOIT-ELDERLY/DISABLED',
       'CRIMINAL SEXUAL ABUSE', 'OTHER CRIME INVOLVING PROPERTY',
       'TRUCK, BUS, MOTOR HOME', 'FORCIBLE ENTRY', 'POSS: CRACK',
       'SIMPLE', 'POSS: HEROIN(WHITE)', 'UNLAWFUL POSS OF 

In [18]:
# Creating an empty dictionary  
CRIMES = {}
# 3. Loop through the list of crimes
for crime in crime_list:
    # I. Save a temp df of just the rows that match the crime
   temp = chicago_df.loc[chicago_df['Description']==crime].copy()
    # II. Resample the temp DataFrame as Daily data (crime counts) 
    # and keep ONLY the .size() 
   temp_res = temp.resample("D").size() 
   #III. Save the temporary DataFrame in the dictionary,
   #using the crime description as the key.
   CRIMES[crime] = temp_res.copy() 
CRIMES.keys()

dict_keys(['FINANCIAL ID THEFT: OVER $300', 'AGG: FINANCIAL ID THEFT', 'PREDATORY', 'AGG SEX ASSLT OF CHILD FAM MBR', 'AGG CRIM SEX ABUSE FAM MEMBER', 'OVER $500', 'AGG CRIMINAL SEXUAL ABUSE', 'FIRST DEGREE MURDER', 'THEFT/RECOVERY: AUTOMOBILE', 'CRIM SEX ABUSE BY FAM MEMBER', 'SEXUAL EXPLOITATION OF A CHILD', 'AUTOMOBILE', 'OTHER VEHICLE OFFENSE', 'FRAUD OR CONFIDENCE GAME', 'SEX ASSLT OF CHILD BY FAM MBR', 'CREDIT CARD FRAUD', '$500 AND UNDER', 'FINANCIAL ID THEFT:$300 &UNDER', 'AGGRAVATED: OTHER', 'NON-AGGRAVATED', 'EMBEZZLEMENT', 'FORGERY', 'HARASSMENT BY ELECTRONIC MEANS', 'BOGUS CHECK', 'CHILD ABUSE', 'FROM BUILDING', 'CONTRIBUTE DELINQUENCY OF A CHILD', 'HARASSMENT BY TELEPHONE', 'FINAN EXPLOIT-ELDERLY/DISABLED', 'CRIMINAL SEXUAL ABUSE', 'OTHER CRIME INVOLVING PROPERTY', 'TRUCK, BUS, MOTOR HOME', 'FORCIBLE ENTRY', 'POSS: CRACK', 'SIMPLE', 'POSS: HEROIN(WHITE)', 'UNLAWFUL POSS OF HANDGUN', 'FALSE POLICE REPORT', 'AGGRAVATED', 'AGGRAVATED VEHICULAR HIJACKING', 'POSS: HEROIN(BLACK 

In [23]:
resampled_df = pd.DataFrame(CRIMES)
resampled_df

,FINANCIAL ID THEFT: OVER $300,AGG: FINANCIAL ID THEFT,PREDATORY,AGG SEX ASSLT OF CHILD FAM MBR,AGG CRIM SEX ABUSE FAM MEMBER,OVER $500,AGG CRIMINAL SEXUAL ABUSE,FIRST DEGREE MURDER,THEFT/RECOVERY: AUTOMOBILE,CRIM SEX ABUSE BY FAM MEMBER,SEXUAL EXPLOITATION OF A CHILD,AUTOMOBILE,OTHER VEHICLE OFFENSE,FRAUD OR CONFIDENCE GAME,SEX ASSLT OF CHILD BY FAM MBR,CREDIT CARD FRAUD,$500 AND UNDER,FINANCIAL ID THEFT:$300 &UNDER,AGGRAVATED: OTHER,NON-AGGRAVATED,EMBEZZLEMENT,FORGERY,HARASSMENT BY ELECTRONIC MEANS,BOGUS CHECK,CHILD ABUSE,...,AGGRAVATED DOMESTIC BATTERY - OTHER FIREARM,CONTRIBUTE TO THE DELINQUENCY OF CHILD,POSSESS - HEROIN (BLACK TAR),OTHER ARSON / EXPLOSIVE INCIDENT,AGGRAVATED OF AN UNBORN CHILD,"ATTEMPT - CYCLE, SCOOTER, BIKE WITH VIN",POSSESSION - EXPLOSIVE / INCENDIARY DEVICE,MANUFACTURE / DELIVER - AMPHETAMINES,INTERFERE WITH EMERGENCY EQUIPMENT,UNLAWFUL SALE - HANDGUN,OF AN UNBORN CHILD,GAME / AMUSEMENT DEVICE,MANUFACTURE / DELIVER - METHAMPHETAMINE,MANUFACTURE / DELIVER - SYNTHETIC MARIJUANA,"AGG. RITUAL MUTILATION - HANDS, FISTS, FEET, SERIOUS INJURY",SELL / ADVERTISE FIREWORKS,"ATTEMPT - CYCLE, SCOOTER, BIKE NO VIN",DISCLOSE DOMESTIC VIOLENCE VICTIM LOCATION,DELIVER CANNABIS TO PERSON UNDER 18,SECOND DEGREE MURDER,POSSESS - HYPODERMIC NEEDLE,"THEFT / RECOVERY - CYCLE, SCOOTER, BIKE NO VIN",DELIVER CONTROLLED SUBSTANCES TO PERSON UNDER 18,POSSESS KEYS OR DEVICE TO COIN MACHINE,POSSESS - LOOK-ALIKE DRUGS
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-01,96.0,7.0,17,28.0,15.0,5,13.0,2,1.0,11.0,2.0,2,1,4,8.0,3,2,27.0,8.0,1,3.0,4,1,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,5.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,2.0,0.0,0,1.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,1.0,0.0,0,0.0,0.0,0,1.0,0,0.0,0.0,0.0,1,0,1,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,1.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,2,0,0,0.0,0,0,0.0,0.0,0,0.0,1,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,NaN,NaN,1,NaN,NaN,77,NaN,2,NaN,NaN,NaN,50,2,6,NaN,4,62,NaN,NaN,3,NaN,2,7,0,1.0,...,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-27,NaN,NaN,0,NaN,NaN,71,NaN,3,NaN,NaN,NaN,52,3,2,NaN,3,50,NaN,NaN,5,NaN,3,8,0,3.0,...,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-28,NaN,NaN,0,NaN,NaN,48,NaN,2,NaN,NaN,NaN,43,3,0,NaN,2,32,NaN,NaN,1,NaN,0,3,0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
## saving to disk
resampled_df.to_csv("Data/Chicago/Chicago_crime_counts.csv")

In [10]:
chicago_df.groupby("Description").size().head()

Description
$300 AND UNDER                         10
$500 AND UNDER                     552169
ABUSE / NEGLECT - CARE FACILITY        38
ABUSE/NEGLECT: CARE FACILITY          133
ADULTRY                                 6
dtype: int64

In [11]:
# II. Resample the temp DataFrame as Daily data (crime counts)
# and keep ONLY the .size()
temp_res = temp.resample("D").size()
temp_res




NameError: name 'temp' is not defined

In [26]:
## testing our saved data
resampled_df = pd.read_csv("Data/Chicago/Chicago_crime_counts.csv", parse_dates=['Datetime'], index_col=0)
resampled_df

,FINANCIAL ID THEFT: OVER $300,AGG: FINANCIAL ID THEFT,PREDATORY,AGG SEX ASSLT OF CHILD FAM MBR,AGG CRIM SEX ABUSE FAM MEMBER,OVER $500,AGG CRIMINAL SEXUAL ABUSE,FIRST DEGREE MURDER,THEFT/RECOVERY: AUTOMOBILE,CRIM SEX ABUSE BY FAM MEMBER,SEXUAL EXPLOITATION OF A CHILD,AUTOMOBILE,OTHER VEHICLE OFFENSE,FRAUD OR CONFIDENCE GAME,SEX ASSLT OF CHILD BY FAM MBR,CREDIT CARD FRAUD,$500 AND UNDER,FINANCIAL ID THEFT:$300 &UNDER,AGGRAVATED: OTHER,NON-AGGRAVATED,EMBEZZLEMENT,FORGERY,HARASSMENT BY ELECTRONIC MEANS,BOGUS CHECK,CHILD ABUSE,...,AGGRAVATED DOMESTIC BATTERY - OTHER FIREARM,CONTRIBUTE TO THE DELINQUENCY OF CHILD,POSSESS - HEROIN (BLACK TAR),OTHER ARSON / EXPLOSIVE INCIDENT,AGGRAVATED OF AN UNBORN CHILD,"ATTEMPT - CYCLE, SCOOTER, BIKE WITH VIN",POSSESSION - EXPLOSIVE / INCENDIARY DEVICE,MANUFACTURE / DELIVER - AMPHETAMINES,INTERFERE WITH EMERGENCY EQUIPMENT,UNLAWFUL SALE - HANDGUN,OF AN UNBORN CHILD,GAME / AMUSEMENT DEVICE,MANUFACTURE / DELIVER - METHAMPHETAMINE,MANUFACTURE / DELIVER - SYNTHETIC MARIJUANA,"AGG. RITUAL MUTILATION - HANDS, FISTS, FEET, SERIOUS INJURY",SELL / ADVERTISE FIREWORKS,"ATTEMPT - CYCLE, SCOOTER, BIKE NO VIN",DISCLOSE DOMESTIC VIOLENCE VICTIM LOCATION,DELIVER CANNABIS TO PERSON UNDER 18,SECOND DEGREE MURDER,POSSESS - HYPODERMIC NEEDLE,"THEFT / RECOVERY - CYCLE, SCOOTER, BIKE NO VIN",DELIVER CONTROLLED SUBSTANCES TO PERSON UNDER 18,POSSESS KEYS OR DEVICE TO COIN MACHINE,POSSESS - LOOK-ALIKE DRUGS
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-01,96.0,7.0,17,28.0,15.0,5,13.0,2,1.0,11.0,2.0,2,1,4,8.0,3,2,27.0,8.0,1,3.0,4,1,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,5.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,2.0,0.0,0,1.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,1.0,0.0,0,0.0,0.0,0,1.0,0,0.0,0.0,0.0,1,0,1,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,1.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,2,0,0,0.0,0,0,0.0,0.0,0,0.0,1,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,NaN,NaN,1,NaN,NaN,77,NaN,2,NaN,NaN,NaN,50,2,6,NaN,4,62,NaN,NaN,3,NaN,2,7,0,1.0,...,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-27,NaN,NaN,0,NaN,NaN,71,NaN,3,NaN,NaN,NaN,52,3,2,NaN,3,50,NaN,NaN,5,NaN,3,8,0,3.0,...,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-28,NaN,NaN,0,NaN,NaN,48,NaN,2,NaN,NaN,NaN,43,3,0,NaN,2,32,NaN,NaN,1,NaN,0,3,0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
resampled_df = resampled_df.resample('D').asfreq()
resampled_df.head(3)


,FINANCIAL ID THEFT: OVER $300,AGG: FINANCIAL ID THEFT,PREDATORY,AGG SEX ASSLT OF CHILD FAM MBR,AGG CRIM SEX ABUSE FAM MEMBER,OVER $500,AGG CRIMINAL SEXUAL ABUSE,FIRST DEGREE MURDER,THEFT/RECOVERY: AUTOMOBILE,CRIM SEX ABUSE BY FAM MEMBER,SEXUAL EXPLOITATION OF A CHILD,AUTOMOBILE,OTHER VEHICLE OFFENSE,FRAUD OR CONFIDENCE GAME,SEX ASSLT OF CHILD BY FAM MBR,CREDIT CARD FRAUD,$500 AND UNDER,FINANCIAL ID THEFT:$300 &UNDER,AGGRAVATED: OTHER,NON-AGGRAVATED,EMBEZZLEMENT,FORGERY,HARASSMENT BY ELECTRONIC MEANS,BOGUS CHECK,CHILD ABUSE,...,AGGRAVATED DOMESTIC BATTERY - OTHER FIREARM,CONTRIBUTE TO THE DELINQUENCY OF CHILD,POSSESS - HEROIN (BLACK TAR),OTHER ARSON / EXPLOSIVE INCIDENT,AGGRAVATED OF AN UNBORN CHILD,"ATTEMPT - CYCLE, SCOOTER, BIKE WITH VIN",POSSESSION - EXPLOSIVE / INCENDIARY DEVICE,MANUFACTURE / DELIVER - AMPHETAMINES,INTERFERE WITH EMERGENCY EQUIPMENT,UNLAWFUL SALE - HANDGUN,OF AN UNBORN CHILD,GAME / AMUSEMENT DEVICE,MANUFACTURE / DELIVER - METHAMPHETAMINE,MANUFACTURE / DELIVER - SYNTHETIC MARIJUANA,"AGG. RITUAL MUTILATION - HANDS, FISTS, FEET, SERIOUS INJURY",SELL / ADVERTISE FIREWORKS,"ATTEMPT - CYCLE, SCOOTER, BIKE NO VIN",DISCLOSE DOMESTIC VIOLENCE VICTIM LOCATION,DELIVER CANNABIS TO PERSON UNDER 18,SECOND DEGREE MURDER,POSSESS - HYPODERMIC NEEDLE,"THEFT / RECOVERY - CYCLE, SCOOTER, BIKE NO VIN",DELIVER CONTROLLED SUBSTANCES TO PERSON UNDER 18,POSSESS KEYS OR DEVICE TO COIN MACHINE,POSSESS - LOOK-ALIKE DRUGS
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-01,96.0,7.0,17,28.0,15.0,5,13.0,2,1.0,11.0,2.0,2,1,4,8.0,3,2,27.0,8.0,1,3.0,4,1,1,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,5.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,2.0,0.0,0,1.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,1.0,0.0,0,0.0,0.0,0,1.0,0,0.0,0.0,0.0,1,0,1,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
ts0 = resampled_df.index[0]
ts0

Timestamp('2001-01-01 00:00:00', freq='D')

In [39]:
# checking the documentation for astimezone
ts0.astimezone?

# 

In [ ]:
## remove time zone from the dt index
df = df.tz_convert(None)
df.head(3)

In [45]:
resampled_df.isna().sum()

FINANCIAL ID THEFT: OVER $300                       3402
AGG: FINANCIAL ID THEFT                             3405
PREDATORY                                              0
AGG SEX ASSLT OF CHILD FAM MBR                      1220
AGG CRIM SEX ABUSE FAM MEMBER                       1235
                                                    ... 
POSSESS - HYPODERMIC NEEDLE                         8215
THEFT / RECOVERY - CYCLE, SCOOTER, BIKE NO VIN      8109
DELIVER CONTROLLED SUBSTANCES TO PERSON UNDER 18    8215
POSSESS KEYS OR DEVICE TO COIN MACHINE              8215
POSSESS - LOOK-ALIKE DRUGS                          8215
Length: 542, dtype: int64

In [46]:
## filling the null values with 0
resampled_df = resampled_df.fillna(0)
resampled_df



,FINANCIAL ID THEFT: OVER $300,AGG: FINANCIAL ID THEFT,PREDATORY,AGG SEX ASSLT OF CHILD FAM MBR,AGG CRIM SEX ABUSE FAM MEMBER,OVER $500,AGG CRIMINAL SEXUAL ABUSE,FIRST DEGREE MURDER,THEFT/RECOVERY: AUTOMOBILE,CRIM SEX ABUSE BY FAM MEMBER,SEXUAL EXPLOITATION OF A CHILD,AUTOMOBILE,OTHER VEHICLE OFFENSE,FRAUD OR CONFIDENCE GAME,SEX ASSLT OF CHILD BY FAM MBR,CREDIT CARD FRAUD,$500 AND UNDER,FINANCIAL ID THEFT:$300 &UNDER,AGGRAVATED: OTHER,NON-AGGRAVATED,EMBEZZLEMENT,FORGERY,HARASSMENT BY ELECTRONIC MEANS,BOGUS CHECK,CHILD ABUSE,...,AGGRAVATED DOMESTIC BATTERY - OTHER FIREARM,CONTRIBUTE TO THE DELINQUENCY OF CHILD,POSSESS - HEROIN (BLACK TAR),OTHER ARSON / EXPLOSIVE INCIDENT,AGGRAVATED OF AN UNBORN CHILD,"ATTEMPT - CYCLE, SCOOTER, BIKE WITH VIN",POSSESSION - EXPLOSIVE / INCENDIARY DEVICE,MANUFACTURE / DELIVER - AMPHETAMINES,INTERFERE WITH EMERGENCY EQUIPMENT,UNLAWFUL SALE - HANDGUN,OF AN UNBORN CHILD,GAME / AMUSEMENT DEVICE,MANUFACTURE / DELIVER - METHAMPHETAMINE,MANUFACTURE / DELIVER - SYNTHETIC MARIJUANA,"AGG. RITUAL MUTILATION - HANDS, FISTS, FEET, SERIOUS INJURY",SELL / ADVERTISE FIREWORKS,"ATTEMPT - CYCLE, SCOOTER, BIKE NO VIN",DISCLOSE DOMESTIC VIOLENCE VICTIM LOCATION,DELIVER CANNABIS TO PERSON UNDER 18,SECOND DEGREE MURDER,POSSESS - HYPODERMIC NEEDLE,"THEFT / RECOVERY - CYCLE, SCOOTER, BIKE NO VIN",DELIVER CONTROLLED SUBSTANCES TO PERSON UNDER 18,POSSESS KEYS OR DEVICE TO COIN MACHINE,POSSESS - LOOK-ALIKE DRUGS
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-01,96.0,7.0,17,28.0,15.0,5,13.0,2,1.0,11.0,2.0,2,1,4,8.0,3,2,27.0,8.0,1,3.0,4,1,1,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-02,5.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,2.0,0.0,0,1.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-03,1.0,0.0,0,0.0,0.0,0,1.0,0,0.0,0.0,0.0,1,0,1,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-04,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-05,1.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,2,0,0,0.0,0,0,0.0,0.0,0,0.0,1,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,0.0,0.0,1,0.0,0.0,77,0.0,2,0.0,0.0,0.0,50,2,6,0.0,4,62,0.0,0.0,3,0.0,2,7,0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-27,0.0,0.0,0,0.0,0.0,71,0.0,3,0.0,0.0,0.0,52,3,2,0.0,3,50,0.0,0.0,5,0.0,3,8,0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-28,0.0,0.0,0,0.0,0.0,48,0.0,2,0.0,0.0,0.0,43,3,0,0.0,2,32,0.0,0.0,1,0.0,0,3,0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Q1: Which district has the most crimes? Which has the least?


In [63]:
df_ts = chicago_df.groupby('District').sum()
df_ts

C:\Users\caill\AppData\Local\Temp\ipykernel_25012\655895210.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ts = chicago_df.groupby('District').sum()


,ID,Arrest,Domestic,Beat,Ward,Latitude,Longitude,Total # of Crimes
District,,,,,,,,
1.0,2322496986567,78756,14318,68576416,5917976.0,1.226763e+07,-2.567347e+07,0.0
2.0,2379563209111,79518,44740,204145780,1722587.0,1.355484e+07,-2.840410e+07,0.0
3.0,2707354769568,91866,75571,117360884,3952735.0,1.517654e+07,-3.182605e+07,0.0
4.0,3085858693218,91209,65595,172476767,3324443.0,1.707978e+07,-3.583572e+07,0.0
5.0,2388726565355,79773,62122,165692687,5944355.0,1.325801e+07,-2.786691e+07,0.0
6.0,3233376140778,108357,80919,262710636,5827025.0,1.762525e+07,-3.699899e+07,0.0
7.0,3031563364512,115833,83742,299658491,6064684.0,1.733186e+07,-3.636565e+07,0.0
8.0,3566584588148,108098,70243,397398116,8089554.0,2.014824e+07,-4.230236e+07,0.0
9.0,2547125176088,94933,45144,330526061,4511038.0,1.456086e+07,-3.052644e+07,0.0
